In [1]:
import json
import numpy as np
import string
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -q accelerate==0.24.1
# !pip install -q huggingface-hub==0.17.3
# !pip install -q safetensors==0.4.0
# !pip install -q tokenizers==0.14.1
# !pip install -q transformers==4.35.0

In [3]:
! pip install -q transformers datasets sentencepiece

!pip install -q safetensors

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q evaluate

!pip install -q parsivar

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import importlib.util
print(importlib.util.find_spec("safetensors") is not None)

True


In [10]:
generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
answer_scores = generated_answer["sequences_scores"]

# Print the generated answers and their scores
for ans, score in zip(generated_answers, answer_scores):
    print(f"Answer: {ans}, Score: {score}")

Answer: امانتش, Score: -0.16079461574554443
Answer: امانت, Score: -0.2284049689769745
Answer: امانش, Score: -0.8540023565292358
Answer: امانتها, Score: -1.363860011100769
Answer: امانه, Score: -1.3642112016677856
Answer: امانته, Score: -1.3867406845092773
Answer: امانتی, Score: -1.45795738697052
Answer: امانات, Score: -1.609614372253418
Answer: امانیت, Score: -1.6619296073913574
Answer: امانتان, Score: -1.6641783714294434
Answer: امان, Score: -1.6769590377807617
Answer: امانشان, Score: -1.7007102966308594
Answer: امانتن, Score: -1.7829428911209106
Answer: امانی, Score: -1.8085150718688965
Answer: اماننه, Score: -1.8488987684249878
Answer: امانتا, Score: -1.966618299484253
Answer: امان اش, Score: -1.9839469194412231
Answer: اماننش, Score: -2.0012831687927246
Answer: امانیش, Score: -2.0279836654663086
Answer: امانمه, Score: -2.0297656059265137


In [4]:
import json
import torch
# from transformers import AutoTokenizer
# from transformers import AutoModelForQuestionAnswering
from transformers import MBart50TokenizerFast
from transformers import MBartForConditionalGeneration
from parsivar import Normalizer
import evaluate

metric = evaluate.load("squad")


model_name_or_path = '/content/drive/MyDrive/mBART_QA/checkpoint-4409'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
model = MBartForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True)

my_normalizer = Normalizer()


with open('/content/drive/MyDrive/extra_translations/ansarian/ansarian_dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  # ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']




  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                max_length=64, top_p=0.95, top_k=50, num_return_sequences=20,
                                output_scores = True, return_dict_in_generate = True, num_beams=20)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["sequences_scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score
    # print(f"Answer: {ans}, Score: {score}")

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_output.append(my_normalizer.normalize(final_answer).strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  # new_ex['answers']['text'] = [qa['answers'][0]['text']]
  new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)

metric.compute(predictions=predicted_answers, references=theoretical_answers)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'exact_match': 60.36585365853659, 'f1': 67.22706155632984}